In [1]:
%pip install llama-cpp-python fire

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import fire
from llama_cpp import Llama
import json
import torch
import pandas as pd
import csv
from clear_text import clear_str

2024-09-29 05:57:07.876079: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 05:57:07.898693: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 05:57:07.905471: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 05:57:07.926704: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-29 05:57:09.103729: W tensorflow/comp

In [ ]:
PROMPT_1_LVL = "Отнеси описание данного видео к 1 или 2 категориям. Если в описании есть имя собственное добавь категорию Массовая культура. \n" \
        "{categories}\n\n" \
        "Текст: Артмеханика. Концерт Сэм Пикар. Концерт Сэм Пикар. \n" \
        "Ответ: Музыка и аудио, События и достопримечательности. \n\n" \
        "Текст: Злые языки | Выпуск 3, Сезон 4 | Марат Башаров. Магические способности актёра. Скандал с Сафроновым. Опекунство над детьми из Сирии. Новая любовь - скоро свадьба? \n" \
        "Ответ: Массовая культура. \n\n" \
        "Текст: ПО РУКАМ #6. В начале выпуска посмотрим, сможет ли пацанка Настя Петрова победить блогера Влада Левского. ПРО-АРМ: Виктора Шевеленко и Байсангур Солтыханов встретятся в реванше профессионалов. ПОП-АРМ: второй полуфинал! Рекордсмен в подъеме на бицепс Андрей Качалин против фитнес-блогера Алексея Шредера. \n" \
        "Ответ: Спорт, Массовая культура \n\n" \
        "Текст: Пацанский клининг. Шоу «ЧистоТачка» | Повелитель ночи | Выпуск 17. Тяпа и Егор бросили вызов нестареющему «повелителю ночи». Смогут ли они подарить ему статус «ЧистоТачки»? \n" \
        "Ответ: Массовая культура \n\n"
PROMPT_2_LVL= "Отнеси описание данного видео к категории. Если не можешь выбрать категорию ответь Нет \n" \
        "{categories}\n\n" \
        "Текст: Миллионы людей в России страдают от абьюзивных отношений. Они подвергаются не только физическому, но и моральному насилию. \n" \
        "Ответ: {random_category}. \n\n" \
        "Текст: Меня зовут Саша Кабаева. Я успешная и независимая девушка. Но совсем недавно я была жертвой абьюза. И я, как никто другой, знаю, как из этого выйти. \n" \
        "Ответ: Нет. \n\n"
categories_for_whisper = ["Наука", ""]
categories_for_video = ["", ""]

In [9]:
with open("data/categories.json") as f:
    categories = json.loads(f.read())
    categories_1_lvl = list(categories.keys())

model_path = "data/model-q4_K.gguf"
n_ctx=8192
top_k=30
top_p=0.9
temperature=0.6
repeat_penalty=1.1

messages_1_lvl = [{"role": "system", "content": PROMPT_1_LVL.format(categories='\n'.join(categories_1_lvl))}]

In [5]:
def define_categories_2_lvl(category_1_lvl):
    if type(category_1_lvl) == dict: 
        return list(category_1_lvl.keys())[0] 
    return category_1_lvl

In [ ]:
def generate_2_lvl_categories(category_1_lvl, model):
    if category_1_lvl in categories_for_whisper:
        text = get_text_from_audio(video_id)
        asnwer = generate_answer(text, model)
        categories_2_lvl = list(map(lambda x: define_categories_2_lvl(x) , categories[category]))
        messages_2_lvl = [{"role": "system", "content": PROMPT_2_LVL.format(categories='\n'.join(categories_2_lvl), random_category=categories_2_lvl[0])}]
        if answer.lower() != "Нет":
            return category_1_lvl + ": " + answer
    else if category_1_lvl in categories_for_video:
        text = get_text_from_video(video_id)
        asnwer = generate_answer(text, model)
        categories_2_lvl = list(map(lambda x: define_categories_2_lvl(x) , categories[category]))
        messages_2_lvl = [{"role": "system", "content": PROMPT_2_LVL.format(categories='\n'.join(categories_2_lvl), random_category=categories_2_lvl[0])}]
        if answer.lower() != "Нет":
            return category_1_lvl + ": " + answer
    return category_1_lvl
        

In [25]:
def generate_answer(text, model, messages):
    prompt = "Текст: {text} \n" \
                "Ответ: "
    messages.append({"role": "user", "content": prompt.format(text=text)})
    answer = ""
    for part in model.create_chat_completion(
        messages,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repeat_penalty=repeat_penalty,
        stream=True,
    ):
        delta = part["choices"][0]["delta"]
        if "content" in delta:
            answer += delta["content"]
            print(delta["content"], end="", flush=True)
    return answer

In [6]:
def preparation_teg(teg):
  if teg.find(":") == -1:
    return teg.strip()
  return teg[:teg.find(":")].strip()

In [7]:
def determine_1_level_categories(x):
  result = x.split(",")
  result = list(map(lambda x: preparation_teg(x), result))
  return result

In [8]:
def iou_metric(ground_truth, predictions):
    iou =  len(set.intersection(set(ground_truth), set(predictions)))
    iou = iou/(len(set(ground_truth).union(set(predictions))))
    return iou

In [9]:
def split_tags(tag_list):
    final_tag_list = []
    for tag in tag_list:
        tags = tag.split(": ")
        if len(tags) == 3:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
            final_tag_list.append(tags[0]+ ": " + tags[1] + ": " + tags[2])
        elif len(tags) == 2:
            final_tag_list.append(tags[0])
            final_tag_list.append(tags[0] + ": " + tags[1])
        elif len(tags) == 1:
            final_tag_list.append(tags[0])
        else:
            print("NOT IMPLEMENTED!!!!", tag)
    return final_tag_list

In [10]:
def find_iou_for_sample_submission(pred_submission, true_submission):
    ground_truth_df = true_submission
    ground_truth_df["tags"] = ground_truth_df["tags"].apply(lambda l: l.split(', '))
    ground_truth_df["tags_split"] = ground_truth_df["tags"].apply(lambda l: split_tags(l))

    predictions_df = pred_submission
    predictions_df["predicted_tags"] = predictions_df["predicted_tags"].apply(lambda l: l.split(', '))
    predictions_df["predicted_tags_split"] = predictions_df["predicted_tags"].apply(lambda l: split_tags(l))
    iou=0
    counter = 0
    for i, row in ground_truth_df.iterrows():
        predicted_tags = predictions_df[predictions_df["video_id"]==row["video_id"]]["predicted_tags_split"].values[0]
        iou_temp=iou_metric(row['tags_split'], predicted_tags)
        iou+=iou_temp
        counter+=1

    return iou/counter

In [ ]:
def write_answer_to_csv(row):
    with open('data/pred_data.csv', 'a', newline='') as csvfile:
        csv.writer(csvfile).writerow(row) 

In [ ]:
def generate_pred_submission(df):
    df_pred = pd.DataFrame(columns=("video_id","title","description","predicted_tags"))
    df_pred.to_csv('data/pred_data.csv', index=False, encoding='utf-8')
    model = Llama(
        model_path="data/model-q4_K.gguf",
        n_ctx=n_ctx,
        n_parts=1,
        verbose=True,
    )
    list_pred = []
    for i, row in df.iterrows():
        text = clear_str(row["title"] + "." + row["description"])
        category_1_lvl = generate_answer(text, model)
        row["predicted_tags"] = category_1_lvl
        category = generate_2_lvl_categories(category_1_lvl, model)
        write_answer_to_csv(row)
        list_pred.append(category)
    pred_submission = df.copy()
    pred_submission["predicted_tags"] = list_pred
    return pred_submission
    

In [26]:
def generate_pred_submission_only_1_lvl(df):
    df_pred = pd.DataFrame(columns=("video_id","title","description","predicted_tags"))
    df_pred.to_csv('data/pred_data.csv')
    model = Llama(
        model_path="data/model-q4_K.gguf",
        n_ctx=n_ctx,
        n_parts=1,
        verbose=True,
    )
    list_pred = []
    for i, row in df.iterrows():
        text = clear_str(row["title"] + "." + row["description"])
        answer = generate_answer(text, model)
        row["predicted_tags"] = answer
        write_answer_to_csv(row)
        list_pred.append(answer)
    pred_submission = df.copy()
    pred_submission["predicted_tags"] = list_pred
    return pred_submission
    

In [27]:
pred_submission = generate_pred_submission_only_1_lvl(df)

llama_model_loader: loaded meta data with 23 key-value pairs and 291 tensors from data/model-q4_K.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = saiga_llama3_8b
llama_model_loader: - kv   2:                          llama.block_count u32              = 32
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.attention.head_count u32              = 32
llama_model_loader: - kv   7:              llama.attention.head_count_kv u32        

Информационные технологии, Массовая культура

llama_perf_context_print:        load time =   70407.42 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   761 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    14 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   75840.07 ms /   775 tokens


NameError: name 'write_answer_to_csv' is not defined

In [16]:
pred_submission

,video_id,title,description,tags,predicted_tags
121,01bd6b00e670655ca7ff484ff55d8ad9,ПО РУКАМ #8,Никита Юрковец и Низами Тагиев откроют этот вы...,"Спорт, Массовая культура",Ответ: Массовая культура
821,acba788b86a3bd559044aadc488455c3,"Команда ""3/21"" в ГрандТуре-2022: заказник ""Леб...",Удивительная красота алтайской зимы не может о...,"Путешествия, События и достопримечательности: ...","Массовая культура, Транспорт"
394,663ac16d23ef52bd261af0a41e6d8f6b,Роман Юнусов и блогер Кирилл Нечаев метают коп...,В новом выпуске шоу «Спортивный Интерес» Рома ...,"Массовая культура, Спорт","Спорт, Массовая культура"
364,a5ca18452c2ad26eeaa2007683840755,Пацанский клининг. Шоу «ЧистоТачка» | Белая бе...,Тяпу и Андрюху «Чернобыля» испугает своим сост...,Массовая культура: Юмор и сатира,Массовая культура
459,8769ca8f5a3e0f53ff89c19e98392d76,"Мама 21 века| 2 сезон, 5 выпуск | Льготы для м...",Многодетным семьям в России оказывается поддер...,"Семья и отношения, Личные финансы: Финансовая ...","Семья и отношения, Личные финансы"
...,...,...,...,...,...
533,8884c062bfa7739bb0bd71bb02fd7893,Байкальская миля 2020: лед и скорость!,На фестивале скорости «Байкальская миля 2020»...,"Транспорт, Спорт: Автогонки, События и достопр...","Транспорт, События и достопримечательности"
191,72ac02e9fa513e91af1c02dd2d79eb6a,НАЧАЛЬНИК ПОЦЕЛОВАЛ ОЛЮ ПДН?,Телеграм канал Начальника: Телеграм канал Оли...,Массовая культура: Юмор и сатира,"Массовая культура, Семья и отношения"
311,54e99123e2326dc0b8d6c4f743da959c,Давай сделаем ЭТО на кухне | Найти любовь в Ин...,"Честно, хлёстко и порой на грани. Влад Чижов с...","Еда и напитки: Кулинария, Массовая культура, С...","Еда и напитки, Семья и отношения"
747,0b73cea7b2dfdd7048dd84b95f8b9b0e,«МузЛофт - Подкаст» с Ириной Нельсон - 1 июля ...,"Подписывайся, чтобы не пропустить!","Массовая культура, Музыка и аудио","Музыка и аудио, Массовая культура"


In [25]:
find_iou_for_sample_submission(pred_submission, df)


0.28507936507936493